# 第三次课程作业——基础矩阵、单应矩阵估计 

## 任务  

- 根据归一化八点法求取基本矩阵
- 根据四点法求取单映矩阵
- 以 images 文件夹里的 **SDC_0480.JPG** 和 **DSC_0481.JPG** 两幅图片为例

<div align=center>
<img src="images/DSC_0480.JPG" width=200>
<img src="images/DSC_0481.JPG" width=200>
</div>

## 示例代码

- **图像信息提取函数（SIFT特征点和EXIF信息）**

In [1]:
class FeatureProcess:
    """
    Simple SIFT feature process
    """

    def __init__(self, image):
        """
        Init
        :param image: (np.ndarray): Image in RGB
        """
        self.image = image
        self.gray = cv2.cvtColor(self.image, cv2.COLOR_BGR2GRAY)
        self.keypoints = None
        self.descriptors = None

    def extract_features(self):
        """
        Extract SIFT features in image.
        :return: (list, np.ndarray): keypoints, descriptors in image
        """
        sift = cv2.SIFT_create()
        keypoints, descriptors = sift.detectAndCompute(self.gray, None)

        if len(keypoints) <= 20:
            return None, None
        else:
            self.keypoints = keypoints
            self.descriptors = descriptors
            return keypoints, descriptors

class PhotoExifInfo:
    """
    Extract photo exif info
    """

    def __init__(self, photo_path):
        """
        init
        :param photo_path: (str): photo path
        """
        self.photo_path = photo_path
        self.focal_length = None
        self.image_width = None
        self.image_length = None
        self.sensor_pixel_size = None

    def get_tags(self):
        """
        Get tags with interested info
        :return: None
        """
        image_content = open(self.photo_path, 'rb')
        tags = exifread.process_file(image_content)
        self.focal_length = float(
            tags['EXIF FocalLength'].values[0].num) / float(tags['EXIF FocalLength'].values[0].den)
        self.image_width = float(tags['EXIF ExifImageWidth'].values[0])
        self.image_length = float(tags['EXIF ExifImageLength'].values[0])
        self.sensor_pixel_size = tags['MakerNote SensorPixelSize']

    def get_intrinsic_matrix(self):
        """
        Get intrinsic matrix of photo's camera
        :return: (np.ndarray): intrinsic matrix K
        """
        K = np.zeros([3, 3])
        dx = self.sensor_pixel_size.values[0].num / self.sensor_pixel_size.values[0].den / self.image_width
        dy = self.sensor_pixel_size.values[1].num / self.sensor_pixel_size.values[1].den / self.image_length
        fu = self.focal_length / dx
        fv = self.focal_length / dy
        u0 = self.image_width / 2
        v0 = self.image_length / 2
        K[0][0] = fu
        K[1][1] = fv
        K[0][2] = u0
        K[1][2] = v0
        K[2][2] = 1
        return K

    def get_area(self):
        """
        Get area of photo
        :return: (int): area
        """
        return int(self.image_width * self.image_length)

    def get_diam(self):
        """
        Get diam of photo
        :return: (int): diam
        """
        return int(max(self.image_width, self.image_length))

def build_img_info(img_root):
    """
    Get info(img,feat,K) from img
    :param img_root: (str): images root
    :return: (list[np.ndarray], list[dict], list[np.ndarray]): info from img
    """
    imgs = []
    feats = []
    K = []
    for i, name in enumerate(os.listdir(img_root)):
        if '.jpg' in name or '.JPG' in name:
            path = os.path.join(img_root, name)
            img = cv2.imread(path)
            imgs.append(img)
            feature_process = FeatureProcess(img)
            kpt, des = feature_process.extract_features()
            photo_info = PhotoExifInfo(path)
            photo_info.get_tags()
            K.append(photo_info.get_intrinsic_matrix())
            A = photo_info.get_area()
            D = photo_info.get_diam()
            feats.append({'kpt': kpt, 'des': des, 'A': A, 'D': D})
    return imgs, feats, K

- **归一化八点法和四点法（基本矩阵和单映矩阵）**

In [2]:
def get_matches(des_query, des_train):
    """
    Match features between query and train
    :param des_query: (np.ndarray): query descriptors
    :param des_train: (np.ndarray): train descriptors
    :return: (list[cv2.DMatch]): Match info
    """
    bf = cv2.BFMatcher(cv2.NORM_L2)
    matches = bf.knnMatch(des_query, des_train, k=2)

    good = []
    for m, m_ in matches:
        # Ratio is 0.6 ,which to remain enough features
        if m.distance < 0.6 * m_.distance:
            good.append(m)
    return good

def get_match_point(p, p_, matches):
    """
    Find matched keypoints
    :param p: (list[cv2.KeyPoint]): query keypoints
    :param p_: (list[cv2.KeyPoint]): train keypoints
    :param matches: (list[cv2.DMatch]): match info between query and train
    :return: (np.ndarray, np.ndarray): matched keypoints between query and train
    """
    points_query = np.asarray([p[m.queryIdx].pt for m in matches])
    points_train = np.asarray([p_[m.trainIdx].pt for m in matches])
    return points_query, points_train

def normalize(pts, T=None):
    """
    normalize points
    :param pts: (np.ndarray): points to be normalized
    :param T: (np.ndarray): IS None means we need to computer T
    :return: (np.ndarray, np.ndarray): normalized points and T
    """
    if T is None:
        u = np.mean(pts, 0)
        d = np.sum(np.sqrt(np.sum(np.power(pts, 2), 1)))
        T = np.array([
            [np.sqrt(2) / d, 0, -(np.sqrt(2) / d * u[0])],
            [0, np.sqrt(2) / d, -(np.sqrt(2) / d * u[1])],
            [0, 0, 1]
        ])
    return homoco_pts_2_euco_pts(np.matmul(T, euco_pts_2_homoco_pts(pts).T).T), T

def homoco_pts_2_euco_pts(pts):
    """
    Homogeneous coordinate to Euclidean coordinates
    :param pts: (np.ndarray): Homogeneous coordinate
    :return: (np.ndarray): Euclidean coordinates
    """
    if len(pts.shape) == 1:
        pts = pts.reshape(1, -1)
    res = pts / pts[:, -1, None]
    return res[:, :-1].squeeze()

def euco_pts_2_homoco_pts(pts):
    """
    Euclidean coordinate to Homogeneous coordinates
    :param pts: (np.ndarray): Euclidean coordinate
    :return: (np.ndarray): Homogeneous coordinates
    """
    if len(pts.shape) == 1:
        pts = pts.reshape(1, -1)
    one = np.ones(pts.shape[0])
    res = np.c_[pts, one]
    return res.squeeze()

def estimate_fundamental(pts1, pts2, num_sample=8):
    n = pts1.shape[0]
    pts_index = range(n)
    sample_index = random.sample(pts_index, num_sample)
    p1 = pts1[sample_index, :]
    p2 = pts2[sample_index, :]
    n = len(sample_index)
    p1_norm, T1 = normalize(p1, None)
    p2_norm, T2 = normalize(p2, None)
    w = np.zeros((n, 9))
    for i in range(n):
        w[i, 0] = p1_norm[i, 0] * p2_norm[i, 0]
        w[i, 1] = p1_norm[i, 1] * p2_norm[i, 0]
        w[i, 2] = p2_norm[i, 0]
        w[i, 3] = p1_norm[i, 0] * p2_norm[i, 1]
        w[i, 4] = p1_norm[i, 1] * p2_norm[i, 1]
        w[i, 5] = p2_norm[i, 1]
        w[i, 6] = p1_norm[i, 0]
        w[i, 7] = p1_norm[i, 1]
        w[i, 8] = 1

    U, sigma, VT = np.linalg.svd(w)
    f = VT[-1, :].reshape(3, 3)
    U, sigma, VT = np.linalg.svd(f)
    sigma[2] = 0
    f = U.dot(np.diag(sigma)).dot(VT)
    f = T2.T.dot(f).dot(T1)
    return f

def estimate_homo(pts1, pts2, num_sample=4):
    n = pts1.shape[0]
    pts_index = range(n)
    sample_index = random.sample(pts_index, num_sample)
    p1 = pts1[sample_index, :]
    p2 = pts2[sample_index, :]
    n = len(sample_index)
    w = np.zeros((n * 2, 9))
    for i in range(n):
        w[2 * i, 0] = p1[i, 0]
        w[2 * i, 1] = p1[i, 1]
        w[2 * i, 2] = 1
        w[2 * i, 3] = 0
        w[2 * i, 4] = 0
        w[2 * i, 5] = 0
        w[2 * i, 6] = -p1[i, 0] * p2[i, 0]
        w[2 * i, 7] = -p1[i, 1] * p2[i, 0]
        w[2 * i, 8] = -p2[i, 0]
        w[2 * i + 1, 0] = 0
        w[2 * i + 1, 1] = 0
        w[2 * i + 1, 2] = 0
        w[2 * i + 1, 3] = p1[i, 0]
        w[2 * i + 1, 4] = p1[i, 1]
        w[2 * i + 1, 5] = 1
        w[2 * i + 1, 6] = -p1[i, 0] * p2[i, 1]
        w[2 * i + 1, 7] = -p1[i, 1] * p2[i, 1]
        w[2 * i + 1, 8] = -p2[i, 1]
    U, sigma, VT = np.linalg.svd(w)
    h = VT[-1, :].reshape(3, 3)
    return h

In [3]:
def build_F_H_pair_match(feats):
    """
    Build F, H, pair and match
    :param feats: (list[dict]): feat of imgs
    :return: (np.ndarray, np.ndarray, dict, dict): F, H, pair of imgs, match of pairs
    """


    pair = dict()
    match = dict()

    for i in range(len(feats)):
        for j in range(i + 1, len(feats)):
            print(i, j)
            matches = get_matches(
                feats[i]['des'], feats[j]['des'])
            pts1, pts2 = get_match_point(
                feats[i]['kpt'], feats[j]['kpt'], matches)
            assert pts1.shape == pts2.shape
            # Need 8 points to estimate models
            if pts1.shape[0] < 8:
                continue

            F_single = estimate_fundamental(pts1, pts2)
            H_single = estimate_homo(pts1, pts2)

            if pts1.shape[0] < 8:
                continue

            pair.update({(i, j): {'pts1': pts1, 'pts2': pts2}})
            match.update({(i, j): {'match': matches}})


    return F_single, H_single, pair, match

## 结果测试

- **导入所需模块**

In [4]:
import numpy as np
import cv2
import random
import os
import exifread

- **读取图像、提取特征点并计算基础矩阵和单应矩阵**

In [6]:
img_root = 'images/'
imgs, feats, K = build_img_info(img_root)
F, H, pair, match = build_F_H_pair_match(feats)
print("The Fundamental Matrix is:\n", F)
print("The Homography Matrix is:\n", H)

0 1
The Fundamental Matrix is:
 [[ 2.04145376e-10  2.36540296e-08 -1.73467054e-05]
 [-2.17166341e-08  6.62880899e-10 -8.21073367e-05]
 [ 1.74208105e-05  7.92261019e-05  4.78852145e-03]]
The Homography Matrix is:
 [[ 1.13554423e-02 -9.44761813e-05 -1.53000794e-01]
 [-4.50809624e-04  1.12960107e-02  9.88028524e-01]
 [-5.55802425e-07  1.25720789e-07  1.15600657e-02]]
